In [30]:
positiveReviewFileName = "D:\\H100DaysOfML\\rt-polaritydata\\rt-polaritydata\\rt-polarity.pos"
negativeReviewFileName = "D:\\H100DaysOfML\\rt-polaritydata\\rt-polaritydata\\rt-polarity.neg"

In [31]:
with open(positiveReviewFileName, 'r') as f:
    positiveReviews = f.readlines()

with open(negativeReviewFileName, 'r') as f:
    negativeReviews = f.readlines()

In [32]:
testTrainingSplitIndex = 2500

testNegativeReviews = negativeReviews[testTrainingSplitIndex+1:]
testPositiveReviews = positiveReviews[testTrainingSplitIndex+1:]

trainingNegativeReviews = negativeReviews[:testTrainingSplitIndex]
trainingPositiveReviews = positiveReviews[:testTrainingSplitIndex]

In [33]:
positiveWordList = [word for line in trainingPositiveReviews for word in line.split()]
negativeWordList = [word for line in trainingNegativeReviews for word in line.split()]
allWordList = [item for sublist in [positiveWordList,negativeWordList] for item in sublist]
vocabulary = list(set(allWordList))

In [34]:
def extract_features(review):
    review_words=set(review)
    features={}
    for word in vocabulary:
        features[word] = (word in review_words)
    return features

In [35]:
negTaggedTrainingReviewList = [{'review':oneReview.split(),'label':'negative'} for oneReview in trainingNegativeReviews]
posTaggedTrainingReviewList = [{'review':oneReview.split(),'label':'positive'} for oneReview in trainingPositiveReviews]
fullTaggedTrainingData = [item for sublist in [negTaggedTrainingReviewList, posTaggedTrainingReviewList]  for item in sublist]
trainingData = [(review['review'],review['label']) for review in fullTaggedTrainingData]

In [36]:
import nltk

In [37]:
trainingFeatures = nltk.classify.apply_features(extract_features, trainingData)
trainedNBClassifier = nltk.NaiveBayesClassifier.train(trainingFeatures)

In [38]:
def naiveBayesSentimentCalculator(review):
    problemInstance = review.split()
    problemFeature = extract_features(problemInstance)
    return trainedNBClassifier.classify(problemFeature)

In [39]:
def getTestReviewSentiments(naiveBayesSentimentCalculator):
    testNegResults = [naiveBayesSentimentCalculator(review) for review in testNegativeReviews]
    testPosResults = [naiveBayesSentimentCalculator(review) for review in testPositiveReviews]
    labelToNum = {'positive':1, 'negative':-1}
    numericNegResults = [labelToNum[x] for x in testNegResults]
    numericPosResults = [labelToNum[x] for x in testPosResults]
    return {'result-on-positive':numericPosResults,'result-on-negative':numericNegResults}

In [40]:
def runDiagnostics(reviewResult):
    positiveReviewsResult = reviewResult['result-on-positive']
    negativeReviewsResult = reviewResult['result-on-negative']
    numTruePositive = sum(x>0 for x in positiveReviewsResult)
    numTrueNegative = sum(x<0 for x in negativeReviewsResult)
    pctTruePositive = float(numTruePositive)/len(positiveReviewsResult)
    pctTrueNegative = float(numTrueNegative)/len(negativeReviewsResult)
    totalAccurate = numTruePositive + numTrueNegative
    total = len(positiveReviewsResult) + len(negativeReviewsResult)
    print("Accuracy on positive reviews= "+"%.2f"%(pctTruePositive*100)+"%")
    print("Accuracy on negative reviews= "+"%.2f"%(pctTrueNegative*100)+"%")
    print("Overall accuracy = " + "%.2f" % (totalAccurate*100/total)+"%")

In [41]:
runDiagnostics(getTestReviewSentiments(naiveBayesSentimentCalculator))

Accuracy on positive reviews= 73.39%
Accuracy on negative reviews= 77.07%
Overall accuracy = 75.23%


In [42]:
#getTestReviewSentiments(naiveBayesSentimentCalculator)